
Lab Exercise 3 : CSE1014 : Foundations of Artificial Intelligence(AI) : Agent Programming

Learning Agent

# Problem Statement

Based on a user's activity, the browser recommends some webpage by by trying to get insights on the user's interest. Here, the browser behaves as a Learning Agent to provide us the most relevant content based on user's interest. 

A webpage maybe associated with multiple topics and the user may search for more than one topic. Here, we consider that the user can perform the following:
1. Browse for a topic represented by "Browse:Topic"
2. Not Interested for a topic based on which the browser has recommended a webpage represented by "NotInterested:Topic"
3. No Action on the browser.

Developing a Learning Agent which will use the browsering activity and ratings for browser recommendations to compute the relevance of a particular webpage and display the webpages in descending order of the relevance. 

<u>Assumption:</u> A user browses for a website only related to one topic. 

# Importing the required modules

In [14]:
import pandas as pd
import random as r
from tabulate import tabulate

# Creating sample data

Sample data for webpages is taken with :
1. An ID for each webpage
2. Topics the webpage is related to.
3. Relevance(Initialised to 0)

Chosen is a sample of 40 webpages related various topics generated randomly.

## Creating the DataFrame

In [15]:
mayAlsoLikeToView = pd.DataFrame(columns=["WebPage ID","Topic","Relevance"])

## Defining the topics for each Webpage

In [16]:
topics = ["Machine Learning","Artificial Intelligence","Python","Javascript","Education",
          "COVID-19","Pandemics","Airlines","Airport","Aviation"]

## Inserting the required data

In [17]:
for i in range(40):
    tpcs = ",".join(set(r.choices(topics,k=r.randint(1,6))))
    mayAlsoLikeToView = mayAlsoLikeToView.append({'WebPage ID':i+1,'Topic':tpcs,'Relevance':0},ignore_index=True)

## Displaying the data

In [18]:
mayAlsoLikeToView = mayAlsoLikeToView.set_index("WebPage ID")
print(tabulate(mayAlsoLikeToView,headers='keys', tablefmt='fancy_grid'))

╒══════════════╤════════════════════════════════════════════════════════════════════╤═════════════╕
│   WebPage ID │ Topic                                                              │   Relevance │
╞══════════════╪════════════════════════════════════════════════════════════════════╪═════════════╡
│            1 │ Javascript,Airlines,Airport                                        │           0 │
├──────────────┼────────────────────────────────────────────────────────────────────┼─────────────┤
│            2 │ COVID-19,Aviation                                                  │           0 │
├──────────────┼────────────────────────────────────────────────────────────────────┼─────────────┤
│            3 │ Education,Python,Airport,Aviation                                  │           0 │
├──────────────┼────────────────────────────────────────────────────────────────────┼─────────────┤
│            4 │ Machine Learning,Airlines,Airport,Artificial Intelligence          │           0 │


# Agent Programming

## Agent Code

In [19]:
class Agent:
    #Constructor to initialise the class variables.
    def __init__(self):
        self.topicRel = dict(zip(topics,[50]*len(topics)))
        self.operation = None
        self.topic = None
        self.df = mayAlsoLikeToView.copy()
    
    #Calculates the Relevance of a webpage as the average of the relevance of individual topics.
    def calculateRelevance(self,x):
        rel = 0
        tp = []
        if "," in x:
            tp = x.split(",")
        else:
            tp = [x]
        for i in tp:
            rel += self.topicRel[i]
        return rel/len(tp)
    
    #Calculates the relevance of every row in the dataframe.
    def interpretInsights(self):
        self.df['Relevance'] = self.df["Topic"].apply(self.calculateRelevance)

    #Moving in the environment and learning from the user activity.
    def move(self,env):
        for i in env.index:
            for j in env.columns:
                if env.at[i,j] != "NoAction":
                   self.operation,self.topic = env.at[i,j].split(":")
                if self.operation == "Browse":
                   self.topicRel[self.topic] += self.topicRel[self.topic]*0.05
                elif self.operation == "NotInterested":
                   self.topicRel[self.topic] -= self.topicRel[self.topic]*0.05
                self.interpretInsights()
    
    #Returns the dictionary of relevance of each topic.
    def getRel(self):
        return self.topicRel

    #Displays the final order in which webpages are displayed.
    def displayWebpages(self):
        self.df = self.df.sort_values(by=['Relevance'],ascending=False)
        print(tabulate(self.df,headers='keys', tablefmt='fancy_grid'))


## Environment Code

The environment is defined as a matrix with row labels as the days of week and the column labels as the hourly times starting from 5pm to 12pm.(7 X 6 matrix)

In [20]:
class Environment:
    #Constructor to initialise the class variables.
    def __init__(self):
        self.browsedTopics = set()
        self.notInterested = set()
        self.days = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
        self.time = []
        for i in range(5,12):
            self.time.append(str(i%12)+"-"+str(i%12+1))
        self.env = pd.DataFrame(columns=self.time,index=self.days)
        self.topics = ["NoAction","Browse","NotInterested"]
        self.topics = ["NoAction","Browse","NotInterested"]
    
    #Creating the environment.
    def createEnv(self):
        for i in self.env.index:
            for j in self.env.columns:
                x = r.choice(self.topics)
                if x != "NoAction":
                   x = x + ":" + r.choice(topics)
                self.env.at[i,j] = x
    
    #Displaying the environment.
    def displayEnv(self):
        tabulate.PRESERVE_WHITESPACE = False
        print(tabulate(self.env,headers='keys', tablefmt='fancy_grid'))

## Driver's Code

In [21]:
Learner = Agent()
e = Environment()
e.createEnv()
e.displayEnv()
Learner.move(e.env)

╒═══════════╤════════════════════════════════╤════════════════════════════════╤═══════════════════════════════════════╤════════════════════════╤══════════════════════════╤═════════════════════════╤══════════════════════════╕
│           │ 5-6                            │ 6-7                            │ 7-8                                   │ 8-9                    │ 9-10                     │ 10-11                   │ 11-12                    │
╞═══════════╪════════════════════════════════╪════════════════════════════════╪═══════════════════════════════════════╪════════════════════════╪══════════════════════════╪═════════════════════════╪══════════════════════════╡
│ Monday    │ NotInterested:COVID-19         │ NoAction                       │ NoAction                              │ NoAction               │ NoAction                 │ Browse:Machine Learning │ Browse:Education         │
├───────────┼────────────────────────────────┼────────────────────────────────┼─────────────────────

## Final Result

In [22]:
Learner.displayWebpages()

╒══════════════╤════════════════════════════════════════════════════════════════════╤═════════════╕
│   WebPage ID │ Topic                                                              │   Relevance │
╞══════════════╪════════════════════════════════════════════════════════════════════╪═════════════╡
│           27 │ Airlines                                                           │     57.5922 │
├──────────────┼────────────────────────────────────────────────────────────────────┼─────────────┤
│           35 │ Airlines,Education                                                 │     57.5202 │
├──────────────┼────────────────────────────────────────────────────────────────────┼─────────────┤
│            7 │ Education,Airport                                                  │     54.9741 │
├──────────────┼────────────────────────────────────────────────────────────────────┼─────────────┤
│           28 │ Airlines,Education,Aviation                                        │     54.9718 │
